
## Overview

TBD

Import Data and merge the three of them

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Load CSVs from DBFS
sales = spark.read.option("header", True).csv("/FileStore/tables/sales_train_validation_2.csv")
calendar = spark.read.option("header", True).csv("/FileStore/tables/calendar__2_.csv")
prices = spark.read.option("header", True).csv("/FileStore/tables/sell_prices_2.csv")


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))


path name size modificationTime dbfs:/FileStore/tables/calendar__2_.csv calendar__2_.csv 103469 1745020618000 dbfs:/FileStore/tables/employees/ employees/ 0 0 dbfs:/FileStore/tables/part_m_00000 part_m_00000 0 1738614237000 dbfs:/FileStore/tables/part_m_00001 part_m_00001 2258028 1738614237000 dbfs:/FileStore/tables/part_m_00002 part_m_00002 1870261 1738614236000 dbfs:/FileStore/tables/part_m_00003 part_m_00003 1558801 1738614236000 dbfs:/FileStore/tables/part_m_00004 part_m_00004 1018863 1738614236000 dbfs:/FileStore/tables/part_m_00005 part_m_00005 103 1738614236000 dbfs:/FileStore/tables/sales_train_validation_2.csv sales_train_validation_2.csv 120007726 1745020617000 dbfs:/FileStore/tables/sell_prices_2.csv sell_prices_2.csv 203395785 1745020630000

In [0]:
from pyspark.sql.functions import col

# Filter full data per category (NO sampling)
foods_full = sales.filter(col("cat_id") == "FOODS")
hobbies_full = sales.filter(col("cat_id") == "HOBBIES")
households_full = sales.filter(col("cat_id") == "HOUSEHOLD")

# Combine all categories into one DataFrame
combined_sales = foods_full.union(hobbies_full).union(households_full)


In [0]:
# melt df
# Get the list of day columns
day_cols = [col_name for col_name in combined_sales.columns if col_name.startswith("d_")]

# ID columns to keep
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# Unpivot (melt)
from pyspark.sql.functions import expr

melted_expr = "stack({}, {})".format(
    len(day_cols),
    ", ".join([f"'{c}', `{c}`" for c in day_cols])
)

sales_long = combined_sales.selectExpr(*id_vars, melted_expr).withColumnRenamed("col0", "d").withColumnRenamed("col1", "sales")


In [0]:
# merge df
# Join with calendar on 'd'
sales_merged = sales_long.join(calendar, on='d', how='left')

# Join with prices on ['store_id', 'item_id', 'wm_yr_wk']
sales_full = sales_merged.join(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

sales_full.show(10)


+--------+---------------+--------+----+--------------------+-----------+---------+--------+-----+----------+---------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+----------+
|store_id|        item_id|wm_yr_wk|   d|                  id|    dept_id|   cat_id|state_id|sales|      date|  weekday|wday|month|year|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|sell_price|
+--------+---------------+--------+----+--------------------+-----------+---------+--------+-----+----------+---------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+----------+
|    CA_1|HOUSEHOLD_1_001|   11101| d_1|HOUSEHOLD_1_001_C...|HOUSEHOLD_1|HOUSEHOLD|      CA|    3|2011-01-29| Saturday|   1|    1|2011|        null|        null|        null|        null|      0|      0|      0|      6.32|
|    CA_1|HOUSEHOLD_1_001|   11101| d_2|HOUSEHOLD_1_001_C...|HOUSEHOLD_1|HOUSEHOLD|      CA|    0|2011-01-30

## **Build all the features in below**

### Seasonality
- Calendar event (event type) - categorical
- Weekend vs weekday - categorical
- SNAP - binary
- Day of week - categorical
- Month - datetime
- Year - datetime

### Sales related
_Check out all mean, max, min, standard deviation stats_
- Moving average (7/28 days) of volume - numerical
- Moving average (7/28 days) of price - numerical
- Absolute volume - numerical
- Absolute price - numerical

### Geographic
- State_id - categorical
- Store_id - categorical

In [0]:
# timing features
from pyspark.sql.functions import col, to_date, when, month, year

# Convert 'date' column from string to date type
sales_full = sales_full.withColumn("date", to_date("date", "yyyy-MM-dd"))

# Sort by id and date (optional: mostly useful before window operations)
# Note: sorting doesn't persist in Spark unless you collect or cache
sales_full = sales_full.orderBy(["id", "date"])

# Add derived time-based features
sales_full = sales_full.withColumn("day_of_week", col("weekday")) \
    .withColumn("is_weekend", when(col("weekday").isin("Saturday", "Sunday"), 1).otherwise(0)) \
    .withColumn("event_type", col("event_type_1")) \
    .withColumn("month", month(col("date"))) \
    .withColumn("year", year(col("date")))


In [0]:
# snap flags
from pyspark.sql.functions import when

# Add snap_flag based on state_id
sales_full = sales_full.withColumn(
    "snap_flag",
    when(col("state_id") == "CA", col("snap_CA"))
    .when(col("state_id") == "TX", col("snap_TX"))
    .when(col("state_id") == "WI", col("snap_WI"))
    .otherwise(0)
)


In [0]:
# sales rolling features
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg, min as spark_min, max as spark_max, stddev, lag

for window_size in [3, 7, 28]:
    window_spec = (
        Window.partitionBy("id")
        .orderBy("date")
        .rowsBetween(-window_size, -1)  # use only past `window_size` days (exclude current row)
    )

    sales_full = sales_full.withColumn(
        f"sales_mean_{window_size}", avg("sales").over(window_spec)
    ).withColumn(
        f"sales_min_{window_size}", spark_min("sales").over(window_spec)
    ).withColumn(
        f"sales_max_{window_size}", spark_max("sales").over(window_spec)
    ).withColumn(
        f"sales_std_{window_size}", stddev("sales").over(window_spec)
    )


In [0]:
# price rolling feature
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg, min as spark_min, max as spark_max, stddev

for window_size in [3, 7, 28]:
    price_window = (
        Window.partitionBy("store_id", "item_id")
        .orderBy("date")
        .rowsBetween(-window_size, -1)  # Past N days only, exclude current row (shifted)
    )

    sales_full = sales_full.withColumn(
        f"price_mean_{window_size}", avg("sell_price").over(price_window)
    ).withColumn(
        f"price_min_{window_size}", spark_min("sell_price").over(price_window)
    ).withColumn(
        f"price_max_{window_size}", spark_max("sell_price").over(price_window)
    ).withColumn(
        f"price_std_{window_size}", stddev("sell_price").over(price_window)
    )


In [0]:
# daily sales and price
from pyspark.sql.functions import col

# Rename sales and sell_price to daily_sales and daily_price for clarity
sales_full = sales_full.withColumn("daily_sales", col("sales"))
sales_full = sales_full.withColumn("daily_price", col("sell_price"))

Food Category Prediction

In [0]:
sales_full.count(), len(sales_full.columns)


In [0]:
foods_df = sales_full.filter(col("cat_id") == "FOODS")
hobbies_df = sales_full.filter(col("cat_id") == "HOBBIES")
households_df = sales_full.filter(col("cat_id") == "HOUSEHOLD")


In [0]:
from pyspark.sql.functions import (
    col, when, lit, dayofweek, month, year
)

# 1️⃣ Absolute price: rename 'sell_price' to 'daily_price'
foods_df = foods_df.withColumn("daily_price", col("sell_price"))

# 2️⃣ Binary SNAP flag (state-specific SNAP)
foods_df = foods_df.withColumn("snap_flag", 
    when(col("state_id") == "CA", col("snap_CA"))
    .when(col("state_id") == "TX", col("snap_TX"))
    .when(col("state_id") == "WI", col("snap_WI"))
    .otherwise(lit(0))
)

# 3️⃣ Weekend indicator: is_weekend (Saturday=7, Sunday=1)
foods_df = foods_df.withColumn("is_weekend", 
    when((dayofweek("date") == 1) | (dayofweek("date") == 7), lit(1)).otherwise(lit(0))
)

# 4️⃣ Add month and year as numeric (if not already added)
foods_df = foods_df.withColumn("month", month(col("date")))
foods_df = foods_df.withColumn("year", year(col("date")))


In [0]:
from pyspark.sql.functions import col
from pyspark.storagelevel import StorageLevel

foods_df = sales_full.filter(col("cat_id") == "FOODS").persist(StorageLevel.DISK_ONLY)
# Trigger persist without full aggregation
foods_df.write.format("noop").mode("overwrite").save()



In [0]:
from pyspark.sql.functions import dayofweek, month, year, when, lit

# Add weekend, month, year if not present
foods_df = foods_df.withColumn("is_weekend", when((dayofweek("date") == 1) | (dayofweek("date") == 7), lit(1)).otherwise(lit(0)))
foods_df = foods_df.withColumn("month", month("date"))
foods_df = foods_df.withColumn("year", year("date"))


In [0]:
foods_df = foods_df.withColumn("snap_flag", 
    when(col("state_id") == "CA", col("snap_CA"))
    .when(col("state_id") == "TX", col("snap_TX"))
    .when(col("state_id") == "WI", col("snap_WI"))
    .otherwise(lit(0))
)


In [0]:
#test on small dataset first!
# 最快提取法，加上 checkpoint + .persist() + .write.format("noop").save() 觸發 execution
#from pyspark.storagelevel import StorageLevel

# Step 1: 選擇必要欄位並 limit
#fast_subset_df = foods_df.select(*final_cols).limit(500000).persist(StorageLevel.MEMORY_AND_DISK)

# Step 2: 強制執行（比 count() 快）：可以改成 noop
#fast_subset_df.write.format("noop").mode("overwrite").save()

# Step 3: 轉成 pandas
#foods_pd = fast_subset_df.toPandas()



In [0]:
# Save hobbies_df as a cleaned Parquet file
hobbies_df.write.mode("overwrite").parquet("dbfs:/tmp/hobbies_cleaned.parquet")
print("✅ hobbies_cleaned.parquet saved.")


In [0]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
import pandas as pd

# 加入 row_id
window = Window.orderBy("id")
foods_df = foods_df.withColumn("row_id", row_number().over(window))

# ✅ 設定你要保留的欄位（加上 event_type_1 和 event_type_2）
selected_features = [c for c in foods_df.columns if 'sales_' in c or 'price_' in c] + [
    'is_weekend', 'snap_flag', 'month', 'year',
    'store_id', 'item_id', 'dept_id', 'state_id',
    'event_type_1', 'event_type_2'
]
final_cols = selected_features + ['sales', 'id', 'date']

# ✅ 分批轉成 pandas 並儲存成 pickle
batch_size = 100_000
total_rows = foods_df.count()
num_batches = (total_rows // batch_size) + 1

for i in range(num_batches):
    start = i * batch_size + 1
    end = (i + 1) * batch_size
    batch_df = foods_df.filter((col("row_id") >= start) & (col("row_id") <= end))
    pdf = batch_df.select(*final_cols).toPandas()
    pdf.to_pickle(f"/dbfs/tmp/foods_pandas_batches/foods_batch_{i}.pkl")
    print(f"✅ Saved foods_batch_{i}.pkl")


---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
File <command-1075219403691499>:27
     25 batch_df = foods_df.filter((col("row_id") >= start) & (col("row_id") <= end))
     26 pdf = batch_df.select(*final_cols).toPandas()
---> 27 pdf.to_pickle(f"/dbfs/tmp/foods_pandas_batches/foods_batch_{i}.pkl")
     28 print(f"✅ Saved foods_batch_{i}.pkl")

File /databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:3026, in NDFrame.to_pickle(self, path, compression, protocol, storage_options)
   2976 """
   2977 Pickle (serialize) object to file.
   2978 
   (...)
   3022 4    4    9
   3023 """  # noqa: E501
   3024 from pandas.io.pickle import to_pickle
-> 3026 to_pickle(
   3027     self,
   3028     path,
   3029     compression=compression,
   3030     protocol=protocol,
   3031     storage_options=storage_options,
   3032 )

File /databricks/python/lib/python3.9/site-package

In [0]:
df = spark.read.parquet("/dbfs/tmp/foods_cleaned.parquet")
df.printSchema()
df.show(5)


root
 |-- store_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- wm_yr_wk: string (nullable = true)
 |-- d: string (nullable = true)
 |-- id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- cat_id: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- date: date (nullable = true)
 |-- weekday: string (nullable = true)
 |-- wday: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- event_name_1: string (nullable = true)
 |-- event_type_1: string (nullable = true)
 |-- event_name_2: string (nullable = true)
 |-- event_type_2: string (nullable = true)
 |-- snap_CA: string (nullable = true)
 |-- snap_TX: string (nullable = true)
 |-- snap_WI: string (nullable = true)
 |-- sell_price: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- sna

In [0]:
from pyspark.sql.functions import col, coalesce, concat_ws, lit

df = df.withColumn("event_type", concat_ws("_", coalesce(col("event_type_1"), lit("missing")),
                                                  coalesce(col("event_type_2"), lit("missing"))))


In [0]:
from pyspark.sql.functions import col, coalesce, concat_ws, lit

# 讀入原始資料
df = spark.read.parquet("/dbfs/tmp/foods_cleaned.parquet")

df = df.withColumn(
    "event_type",
    concat_ws("_", 
              coalesce(col("event_type_1"), lit("missing")),
              coalesce(col("event_type_2"), lit("missing")))
)
df.write.mode("overwrite").parquet("/dbfs/tmp/foods_cleaned.parquet")



In [0]:
#轉換成pandas
df = spark.read.parquet("/dbfs/tmp/foods_cleaned.parquet")

selected_cols = [
    'sales', 'id', 'date', 'event_type', 'weekday', 'is_weekend', 'snap_flag',
    'month', 'year', 'state_id', 'store_id', 'item_id', 'dept_id'
] + [c for c in df.columns if 'sales_' in c or 'price_' in c]

from pyspark.storagelevel import StorageLevel

subset_df = df.select(*selected_cols).limit(500_000).persist(StorageLevel.MEMORY_AND_DISK)
subset_df.write.format("noop").mode("overwrite").save()
foods_pd = subset_df.toPandas()




In [0]:
import os
os.makedirs("/dbfs/tmp/foods_pandas_batches", exist_ok=True)


In [0]:
import os

existing_files = os.listdir("/dbfs/tmp/foods_pandas_batches/")
completed_batches = set()

for fname in existing_files:
    if fname.startswith("foods_batch_") and fname.endswith(".pkl"):
        idx = int(fname.split("_")[-1].split(".")[0])
        completed_batches.add(idx)


In [0]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import os

# Reload full DataFrame
df = spark.read.parquet("/dbfs/tmp/foods_cleaned.parquet")

# Step 1: Recalculate batch info
batch_size = 500_000
total_rows = df.count()
num_batches = (total_rows // batch_size) + 1

# Step 2: Check existing batch files
existing_files = os.listdir("/dbfs/tmp/foods_pandas_batches/")
completed_batches = set()

for fname in existing_files:
    if fname.startswith("foods_batch_") and fname.endswith(".pkl"):
        idx = int(fname.split("_")[-1].split(".")[0])
        completed_batches.add(idx)

print(f"✅ Completed batches: {sorted(completed_batches)}")
print(f"🔄 Total to process: {num_batches}")


✅ Completed batches: []
🔄 Total to process: 55


In [0]:
import pandas as pd
import os
from pyspark.sql.functions import col

# Output directory
output_dir = "/dbfs/tmp/foods_pandas_batches/"
os.makedirs(output_dir, exist_ok=True)

# Load base DataFrame again (if needed)
df = spark.read.parquet("/dbfs/tmp/foods_cleaned.parquet")

# Get total rows & setup batching
batch_size = 300_000
total_rows = df.count()
num_batches = (total_rows // batch_size) + 1

# Identify already saved batches
existing_files = os.listdir(output_dir)
completed_batches = set(
    int(f.split("_")[-1].split(".")[0])
    for f in existing_files if f.startswith("foods_batch_") and f.endswith(".pkl")
)

# Loop through batches
for i in range(num_batches):
    if i in completed_batches:
        print(f"✅ Skipping already saved batch {i}")
        continue

    print(f"🚀 Processing batch {i+1}/{num_batches}...")
    
    start = i * batch_size + 1
    end = (i + 1) * batch_size
    
    batch_df = df.filter((col("row_id") >= start) & (col("row_id") <= end))
    batch_pd = batch_df.toPandas()
    
    batch_pd.to_pickle(f"{output_dir}/foods_batch_{i}.pkl")
    print(f"✅ Saved batch {i} with {len(batch_pd)} rows")

print("🎉 All batches processed.")


✅ Skipping already saved batch 0
✅ Skipping already saved batch 1
✅ Skipping already saved batch 2
✅ Skipping already saved batch 3
✅ Skipping already saved batch 4
🚀 Processing batch 6/92...
✅ Saved batch 5 with 300000 rows
🚀 Processing batch 7/92...
✅ Saved batch 6 with 300000 rows
🚀 Processing batch 8/92...
✅ Saved batch 7 with 300000 rows
🚀 Processing batch 9/92...
✅ Saved batch 8 with 300000 rows
🚀 Processing batch 10/92...
✅ Saved batch 9 with 300000 rows
🚀 Processing batch 11/92...
✅ Saved batch 10 with 300000 rows
🚀 Processing batch 12/92...
✅ Saved batch 11 with 300000 rows
🚀 Processing batch 13/92...
✅ Saved batch 12 with 300000 rows
🚀 Processing batch 14/92...
✅ Saved batch 13 with 300000 rows
🚀 Processing batch 15/92...
✅ Saved batch 14 with 300000 rows
🚀 Processing batch 16/92...
✅ Saved batch 15 with 300000 rows
🚀 Processing batch 17/92...
✅ Saved batch 16 with 300000 rows
🚀 Processing batch 18/92...
✅ Saved batch 17 with 300000 rows
🚀 Processing batch 19/92...
✅ Saved ba

Encoding for LSTM

In [0]:
selected_features = [
    'event_type', 'weekday', 'is_weekend', 'snap_flag',
    'month', 'year', 'state_id', 'store_id', 'item_id', 'dept_id'
] + [c for c in df.columns if 'sales_' in c or 'price_' in c]


Download to local file for less burden

In [0]:
# ✅ Use local filesystem path for Python operations
local_dir = "/dbfs/tmp/foods_pandas_batches"   # without trailing slash

# ✅ Create directory if not exists
os.makedirs(local_dir, exist_ok=True)



In [0]:
existing_files = os.listdir(local_dir)
completed_batches = set(
    int(f.split("_")[-1].split(".")[0])
    for f in existing_files if f.startswith("foods_batch_") and f.endswith(".pkl")
)

batch_pd.to_pickle(f"{local_dir}/foods_batch_{i}.pkl")


In [0]:
import os

local_dir = "/dbfs/tmp/foods_pandas_batches"
os.makedirs(local_dir, exist_ok=True)

existing_files = os.listdir(local_dir)
completed_batches = set(
    int(f.split("_")[-1].split(".")[0])
    for f in existing_files if f.startswith("foods_batch_") and f.endswith(".pkl")
)


In [0]:
import os

folder = "/dbfs/tmp/foods_pandas_batches"
files = sorted([f for f in os.listdir(folder) if f.endswith(".pkl")])

for f in files:
    displayHTML(f'<a href="/files/tmp/foods_pandas_batches/{f}" target="_blank" download>{f}</a>')


foods_batch_0.pkl

foods_batch_1.pkl

foods_batch_10.pkl

foods_batch_11.pkl

foods_batch_12.pkl

...

foods_batch_88.pkl

foods_batch_89.pkl

foods_batch_9.pkl

foods_batch_90.pkl

foods_batch_91.pkl

In [0]:
import shutil

# Step 1: Zip the folder (saved to local driver fs: /tmp/)
local_zip_path = "/tmp/foods_pandas_batches.zip"
shutil.make_archive("/tmp/foods_pandas_batches", 'zip', "/dbfs/tmp/foods_pandas_batches")

# Step 2: Move to DBFS-accessible path
dbutils.fs.mv("file:/tmp/foods_pandas_batches.zip", "dbfs:/FileStore/foods_pandas_batches.zip")

print("✅ Zip moved to FileStore. Ready to download!")


✅ Zip moved to FileStore. Ready to download!


Now do the same thing for Households